In [123]:
import numpy as np
import torch.nn as nn
import torch
import numpy as np
from transformers import AutoTokenizer
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [124]:
file_name = "nudity_train1000.pkl"
data = pd.read_pickle(f'training_pkl/{file_name}')
print(data.head)
print(data.columns)

<bound method NDFrame.head of        IMDb_id                                                url None Mild  \
0    tt0111438  https://www.imdb.com/title/tt0111438/parentalg...    2   14   
1    tt0349080  https://www.imdb.com/title/tt0349080/parentalg...    5    4   
2    tt0088680  https://www.imdb.com/title/tt0088680/parentalg...   11   47   
3    tt0066065  https://www.imdb.com/title/tt0066065/parentalg...    3    4   
4    tt0065481  https://www.imdb.com/title/tt0065481/parentalg...    0    5   
..         ...                                                ...  ...  ...   
995  tt0112857  https://www.imdb.com/title/tt0112857/parentalg...    5   14   
996  tt0091886  https://www.imdb.com/title/tt0091886/parentalg...    1    7   
997  tt1228987  https://www.imdb.com/title/tt1228987/parentalg...   11   38   
998  tt0239986  https://www.imdb.com/title/tt0239986/parentalg...    7    3   
999  tt0068732  https://www.imdb.com/title/tt0068732/parentalg...    0   14   

    Moderate Severe  

In [125]:
# Pobranie kolumny 'IMDb_id' jako zmienna X
X = data['IMDb_id']
print(X)
# Pobranie kolumn 'None', 'Mild', 'Moderate', 'Severe' jako zmienna y
y = data[['None', 'Mild', 'Moderate', 'Severe']]
y =  y.astype(float)

0      tt0111438
1      tt0349080
2      tt0088680
3      tt0066065
4      tt0065481
         ...    
995    tt0112857
996    tt0091886
997    tt1228987
998    tt0239986
999    tt0068732
Name: IMDb_id, Length: 1000, dtype: object


In [126]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train.dtypes

None        float64
Mild        float64
Moderate    float64
Severe      float64
dtype: object

In [128]:
X_train = [torch.tensor(tokenizer.encode(text)).float().unsqueeze(0) for text in X_train]
X_test = [torch.tensor(tokenizer.encode(text)).float().unsqueeze(0) for text in X_test]
y_train = [torch.tensor(y).float().unsqueeze(0) for y in y_train.values]
y_test = [torch.tensor(y).float().unsqueeze(0) for y in y_test.values]

In [129]:
X_train[0].size()

torch.Size([1, 7])

In [130]:
embed_dim = len(X[0])
m = nn.Linear(embed_dim, 5)
input = torch.randn(1, embed_dim)
output = m(input)
print(output.size())

torch.Size([1, 5])


In [131]:
#logistic regression
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        out = self.linear(x)
        return out

In [132]:
X_train[0].size()

torch.Size([1, 7])

In [133]:
y_train[0].size()

torch.Size([1, 4])

In [134]:
#now build a model where input will be X_train[0] and output will be y_train[0]
model = LinearRegression(7, 4)

In [135]:
#loss function
criterion = nn.MSELoss()
#optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.000001)

In [136]:
model.forward(X_train[0])

tensor([[ 7725.8228,  4486.2490,  3660.6348, -3169.0393]],
       grad_fn=<AddmmBackward0>)

In [137]:
#train the model
num_epochs = 100
for epoch in range(num_epochs):
    for i in range(len(X_train)):
        optimizer.zero_grad()
        outputs = model(X_train[i])
        loss = criterion(outputs, y_train[i])
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/100], Loss: 25686316.0000


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x8 and 7x4)